In [ ]:
%%sh
cat > pi-mpi.c <<EOF
#include <mpi.h>
#include <stdio.h>

double funkcja(double x)
{
    return x*x;
}

int main(int argc, char *argv[])
{
    //tworzenie zmiennych globalnych
    int i,np,lp;
    int tag=50;
    MPI_Status status;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &np);
    MPI_Comm_size(MPI_COMM_WORLD, &lp);
    
    //jesli proces ma numer 0 to:
    if(np ==0)
    {
      //tworzenie zmiennych lokalnych
      double dx,xp,xk,pole=0,s=0;
      
      printf("\n Obliczenie calki oznaczonej metoda prostakątów trapezów dla funkcji f(x)  =  x * x < = >  f(X) =  x ^ 2 ");
      //Poczatek przedzialu calkowania xp
      xp = 1.0;
      //koniec przedzialu calkowania xk
      xk = 4.0;
      //obliczanie wartosci dx
      dx = (xk-xp)/(double)(lp-1.0);

      //wysylanie wartosci ci xp i xk oraz dx do poszczegolnych procesow

      for(i=1; i<lp; i++)
      {
        MPI_Send(&xp, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD);
        MPI_Send(&xk, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD);
        MPI_Send(&dx, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD);
      }

      //odbieranie oraz sumowanie wynikow z wszystkich procesow

      for(i=1; i<lp; i++)
      {
        MPI_Recv(&s, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
      
       
        pole +=s;
        printf("%1f\n"  , s) ; 
      }

      //wypisanie wartosci pola
      
      printf("\nPole pod calka wynosi = %1f\n", pole);
    }

    if(np != 0)
    {
      //tworzenie zmiennych lokalnych
      double s1,dx1,xp1,xk1;
      double np2=np;
      //odbieranie wartosci od procesu 0
      MPI_Recv(&xp1, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD, &status);
      MPI_Recv(&xk1, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD, &status);
      MPI_Recv(&dx1, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD, &status);

      //obliczanie pola prostokąta dla podanej funkcji
      // Zmien we wzorze 
   
      // dla prostokata
      //      s1 = funkcja(  xp1 +  np * dx1) * dx1;
      //dla trapezów 

            s1 =  ( funkcja(  xp1  +  (np - 1 )  * dx1)  + funkcja(  xp1  +  np * dx1)  ) * 0.5 * dx1  ; 

      // printf("%1f\n"  , s1) ; 
      //wysylanie pola prostokata do procesu 0
      MPI_Send(&s1, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
    }
    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 40 --allow-run-as-root a.out



 Obliczenie calki oznaczonej metoda prostakątów trapezów dla funkcji f(x)  =  x * x < = >  f(X) =  x ^ 2 0.083068
0.095812
0.109467
0.124033
0.139508
0.155894
0.173191
0.191397
0.210514
0.230542
0.251479
0.273327
0.296086
0.319754
0.344333
0.369822
0.396222
0.423532
0.451752
0.480883
0.510924
0.541875
0.573737
0.606509
0.640191
0.674784
0.710287
0.746700
0.784024
0.822258
0.861402
0.901457
0.942421
0.984297
1.027082
1.070778
1.115385
1.160901
1.207328

Pole pod calka wynosi = 21.002959
